# Hava Kirliliği / Air Pollution

In [ ]:
# Gerekli kütüphaneyi kuralım
!pip install earthaccess

import earthaccess
import os
from pathlib import Path
from datetime import datetime

# --- AYARLAR ---

# Hangi yılları indirmek istiyoruz?
HEDEFLENEN_YILLAR = [2020,2021,2022,2023,2024,2025]

# Hangi veri setlerini indireceğiz? (Daha önce test ettiklerimiz)
# ('NASA Veri Seti Kısa Adı', 'Versiyon Numarası')
VERI_SETLERI = [
    ('OMNO2d', '003'),                          # OMI-Aura NO₂ verisi (.he5)
]

# Verilerin indirileceği ana klasör
INDIRME_KLASORU = "NASA_Verileri_2"

# --- KOD BAŞLANGICI ---

print("NASA Earthdata sistemine giriş yapılıyor...")
try:
    earthaccess.login()
    print("✅ Giriş başarılı.")
except Exception as e:
    print(f"❌ Giriş başarısız. Lütfen yukarıdaki .netrc oluşturma adımını kontrol edin. Hata: {e}")
    # Giriş başarısız olursa kodu burada durdur.
    exit()

# Ana indirme klasörünü oluştur
Path(INDIRME_KLASORU).mkdir(exist_ok=True)

# Bugünün tarihini al (2025 yılı için bitiş tarihi olarak kullanılacak)
bugun = datetime.now()

for short_name, version in VERI_SETLERI:
    for yil in HEDEFLENEN_YILLAR:
        print("\n" + "="*50)
        print(f"İşlem başlıyor: Veri Seti='{short_name}', Yıl='{yil}'")
        
        # İndirme için alt klasör oluştur (örn: NASA_Verileri/OMNO2d/2020)
        hedef_klasor = Path(INDIRME_KLASORU) / short_name / str(yil)
        hedef_klasor.mkdir(parents=True, exist_ok=True)
        print(f"Dosyalar '{hedef_klasor}' klasörüne indirilecek.")

        # Zaman aralığını belirle
        baslangic_tarihi = f"{yil}-01-01"
        if yil == bugun.year:
            # Eğer yıl 2025 ise, bitiş tarihi bugün olsun
            bitis_tarihi = bugun.strftime("%Y-%m-%d")
        else:
            # Diğer yıllar için tüm yılı kapsa
            bitis_tarihi = f"{yil}-12-31"
            
        print(f"Zaman Aralığı: {baslangic_tarihi} -> {bitis_tarihi}")

        # NASA sunucularında veri arama
        try:
            sonuclar = earthaccess.search_data(
                short_name=short_name,
                version=version,
                temporal=(baslangic_tarihi, bitis_tarihi),
            )
            
            if not sonuclar:
                print("⚠️ Bu tarih aralığı için hiç dosya bulunamadı.")
                continue # Bir sonraki yıla veya veri setine geç

            print(f"✅ {len(sonuclar)} adet dosya bulundu. İndirme işlemi başlıyor...")
            
            # Bulunan dosyaları indir
            earthaccess.download(sonuclar, local_path=str(hedef_klasor))
            print(f"✨ '{short_name}' için {yil} yılı verileri başarıyla indirildi.")

        except Exception as e:
            print(f"❌ '{short_name}' için {yil} yılında veri aranırken/indirilirken bir hata oluştu: {e}")

print("\n" + "="*50)
print("Tüm indirme işlemleri tamamlandı!")

In [ ]:
!pip install rioxarray

import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as rxr
from pathlib import Path
from tqdm import tqdm
import re
import numpy as np # Koordinatları oluşturmak için eklendi

# --- AYARLAR BÖLÜMÜ ---
VERI_KLASORU = "NASA_Verileri/OMNO2d/2020"
GEOJSON_DOSYASI = "tr-cities.json"
CIKTI_CSV_DOSYASI = "sehir_bazli_no2_skorlari_2020.csv"

# Verinin bulunduğu tek grup yolu
HDF5_DATA_GROUP = 'HDFEOS/GRIDS/ColumnAmountNO2/Data Fields'

DEGISKEN_ADLARI_LISTESI = [
    'ColumnAmountNO2TropCloudScreened',
    'ColumnAmountNO2Trop'
]

# --- KOD BAŞLANGICI ---

def final_skorlari_hesapla():
    print(f"Yerel '{GEOJSON_DOSYASI}' dosyasından il sınırları okunuyor...")
    try:
        iller_gdf = gpd.read_file(GEOJSON_DOSYASI)
        iller_gdf = iller_gdf[['name', 'geometry']].rename(columns={'name': 'sehir'})
        print(f"✅ {len(iller_gdf)} adet il başarıyla yüklendi.")
    except Exception as e:
        print(f"❌ HATA: '{GEOJSON_DOSYASI}' dosyası okunamadı. -> {e}")
        return

    veri_yolu = Path(VERI_KLASORU)
    he5_dosyalari = sorted(list(veri_yolu.glob('*.he5')))
    if not he5_dosyalari:
        print(f"❌ HATA: '{veri_yolu}' klasöründe dosya bulunamadı.")
        return
        
    print(f"İşlenmek üzere {len(he5_dosyalari)} adet dosya bulundu.")

    tum_sonuclar = []

    for dosya_path in tqdm(he5_dosyalari, desc="Günlük Veriler İşleniyor"):
        try:
            # === YENİ VE KESİN YÖNTEM ===
            # 1. Sadece veri içeren grubu aç
            ds_data = xr.open_dataset(dosya_path, engine='h5netcdf', group=HDF5_DATA_GROUP)
            
            # 2. Akıllı değişken seçimi
            kullanilacak_degisken = next((var for var in DEGISKEN_ADLARI_LISTESI if var in ds_data.data_vars), None)
            if not kullanilacak_degisken:
                continue

            rds = ds_data[kullanilacak_degisken]

            # 3. Koordinatları sıfırdan oluştur ve veriye ata
            # Boyutları yeniden adlandır
            rds = rds.rename({rds.dims[0]: 'y', rds.dims[1]: 'x'})
            
            # Global grid için koordinat dizilerini oluştur
            # 1440 piksel -180'den +180'e (boylam), 720 piksel +90'dan -90'a (enlem)
            lon_coords = np.linspace(-180 + (0.25/2), 180 - (0.25/2), 1440)
            lat_coords = np.linspace(90 - (0.25/2), -90 + (0.25/2), 720)
            
            # Oluşturulan koordinatları veriye ata
            rds = rds.assign_coords(x=lon_coords, y=lat_coords)
            
            # Coğrafi referans sistemini belirt
            rds.rio.write_crs("epsg:4326", inplace=True)

            # === ESKİ KOD DEVAM EDİYOR ===
            tarih_str_match = re.search(r'_(\d{4}m\d{4})_', str(dosya_path))
            if not tarih_str_match: continue
            tarih = pd.to_datetime(tarih_str_match.group(1).replace('m', '-'), format='%Y-%m%d').strftime('%Y-%m-%d')
            
            iller_gdf_proj = iller_gdf.to_crs(rds.rio.crs)

            for index, il in iller_gdf_proj.iterrows():
                sehir_adi = il['sehir']
                try:
                    clipped = rds.rio.clip([il['geometry']], drop=True, all_touched=True)
                    ortalama_skor = float(clipped.mean())
                    
                    if pd.notna(ortalama_skor) and ortalama_skor > 0:
                        tum_sonuclar.append({
                            "sehir": sehir_adi,
                            "tarih": tarih,
                            "no2_skoru": ortalama_skor
                        })
                except Exception:
                    # Bu şehir için veri bulunamazsa (örn. okyanusa denk gelirse) atla
                    continue
        except Exception:
            # print(f"\n⚠️ HATA: '{dosya_path.name}' işlenirken genel bir sorun oluştu. Atlanıyor. -> {e}")
            continue

    if not tum_sonuclar:
        print("❌ Üzgünüm, yine hiçbir dosya başarıyla işlenemedi. Veri yapısında beklenmedik bir durum olabilir.")
        return

    print("\nTüm veriler işlendi. CSV dosyası oluşturuluyor...")
    sonuc_df = pd.DataFrame(tum_sonuclar)
    sonuc_df['no2_skoru'] = sonuc_df['no2_skoru'] * 1e15
    sonuc_df.to_csv(CIKTI_CSV_DOSYASI, index=False, encoding='utf-8')
    print(f"✨ Başarılı! Sonuçlar '{CIKTI_CSV_DOSYASI}' dosyasına kaydedildi.")
    print("\nCSV Dosyasının ilk 5 satırı:")
    print(sonuc_df.head())


if __name__ == '__main__':
    final_skorlari_hesapla()

# Sıcaklık Gündüz / Temperature Day

In [ ]:
!pip install earthaccess

import earthaccess
from pathlib import Path
from datetime import datetime

# --- AYARLAR ---

# Veri Seti Bilgileri
VERI_SETI_KISA_ADI = 'VNP21A1D'
VERSIYON = '002'
YIL = 2021 # <-- Artık buraya yazdığın yıl ne ise, sadece o indirilecek.

# Türkiye'yi içine alan coğrafi kutu
TURKIYE_BOUNDING_BOX = (25, 35, 45, 43)

# Verilerin indirileceği klasör
INDIRME_KLASORU = f"NASA_Verileri/{VERI_SETI_KISA_ADI}/{YIL}"

# --- KOD BAŞLANGICI ---

print("NASA Earthdata sistemine giriş yapılıyor...")
try:
    auth = earthaccess.login()
    print("✅ Giriş başarılı.")
except Exception as e:
    print(f"❌ Giriş başarısız. Lütfen .netrc dosyasını kontrol edin. Hata: {e}")
    exit()

Path(INDIRME_KLASORU).mkdir(parents=True, exist_ok=True)
print(f"Dosyalar '{INDIRME_KLASORU}' klasörüne indirilecek.\n")

# === DÜZELTME BURADA: Bitiş tarihini YIL'a göre doğru ayarlayan mantık ===
baslangic_tarihi = f"{YIL}-01-01"
bugun = datetime.now()

if YIL < bugun.year:
    # Eğer geçmiş bir yılı arıyorsak, bitiş tarihi o yılın sonu (31 Aralık) olmalı.
    bitis_tarihi = f"{YIL}-12-31"
else:
    # Eğer içinde bulunduğumuz yılı (2025) arıyorsak, bitiş tarihi bugün olsun.
    bitis_tarihi = bugun.strftime("%Y-%m-%d")
# === DÜZELTME SONU ===

print(f"Veri Seti: {VERI_SETI_KISA_ADI}, Yıl: {YIL}")
print(f"Zaman Aralığı: {baslangic_tarihi} -> {bitis_tarihi}") # Bu satır artık doğru aralığı gösterecek
print(f"Bölge: Türkiye ({TURKIYE_BOUNDING_BOX})")
print("Veriler aranıyor...")

try:
    sonuclar = earthaccess.search_data(
        short_name=VERI_SETI_KISA_ADI,
        version=VERSIYON,
        temporal=(baslangic_tarihi, bitis_tarihi),
        bounding_box=TURKIYE_BOUNDING_BOX
    )
    
    if sonuclar:
        print(f"✅ {len(sonuclar)} adet dosya bulundu. İndirme işlemi başlıyor...")
        print("❗ Bu işlem uzun sürebilir, lütfen sabırlı olun...")
        
        earthaccess.download(sonuclar, local_path=INDIRME_KLASORU)
        
        print(f"\n✨ İndirme tamamlandı! {len(sonuclar)} dosya '{INDIRME_KLASORU}' klasörüne indirildi.")

    else:
        print("⚠ Belirtilen tarih ve bölge için hiç dosya bulunamadı.")

except Exception as e:
    print(f"❌ Arama veya indirme sırasında bir hata oluştu: {e}")

In [ ]:
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as rxr
from pathlib import Path
from tqdm import tqdm
import re
import numpy as np

# --- AYARLAR ---
# Test modunu aktive et ve işlenecek dosya sayısını belirle
TEST_MODU = False
TEST_DOSYA_SAYISI = 50 # Hızlı bir test için sadece 5 dosya

YIL = 2021
VERI_KLASORU = f"NASA_Verileri/VNP21A1D/{YIL}"
GEOJSON_DOSYASI = "tr-cities.json"

print(VERI_KLASORU)
# Çıktı dosya adını test moduna göre ayarla
if TEST_MODU:
    CIKTI_CSV_DOSYASI = f"sehir_bazli_sicaklik_skorlari_{YIL}_TEST.csv"
else:
    CIKTI_CSV_DOSYASI = f"sehir_bazli_sicaklik_skorlari_{YIL}_gunduz.csv"

# h5 dosyası içindeki doğru grup ve değişken adları
HDF5_PARENT_GROUP = 'HDFEOS/GRIDS/VIIRS_Grid_Daily_1km_LST21'
HDF5_DATA_SUBGROUP = 'Data Fields'
HDF5_DEGISKENI = 'LST_1KM'

# NASA VIIRS/MODIS verilerinin kullandığı standart Sinusoidal Projeksiyon
SINUSOIDAL_PROJ = "+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs"

# --- KOD BAŞLANGICI ---

def skorlari_hesapla_test():
    iller_gdf = gpd.read_file(GEOJSON_DOSYASI)
    iller_gdf = iller_gdf[['name', 'geometry']].rename(columns={'name': 'sehir'})
    
    veri_yolu = Path(VERI_KLASORU)
    h5_dosyalari = sorted(list(veri_yolu.glob('*.h5')))
    
    # Test modu aktifse, dosya listesini kısalt
    if TEST_MODU:
        h5_dosyalari = h5_dosyalari[:TEST_DOSYA_SAYISI]
        print(f"🐞 TEST MODU AKTİF: Sadece ilk {len(h5_dosyalari)} dosya işlenecek.")
    
    print(f"İşlenmek üzere {len(h5_dosyalari)} adet sıcaklık dosyası seçildi.")
    print("İşlem başlıyor...")

    tum_sonuclar = []

    for dosya_path in tqdm(h5_dosyalari, desc="Günlük Sıcaklık Verileri İşleniyor"):
        try:
            data_group_path = f"{HDF5_PARENT_GROUP}/{HDF5_DATA_SUBGROUP}"
            ds_data = xr.open_dataset(dosya_path, engine='h5netcdf', group=data_group_path)
            ds_coords = xr.open_dataset(dosya_path, engine='h5netcdf', group=HDF5_PARENT_GROUP)

            lst_data = ds_data[HDF5_DEGISKENI]
            
            lst_data = lst_data.rename({lst_data.dims[0]: 'y', lst_data.dims[1]: 'x'})
            lst_data = lst_data.assign_coords({"y": ds_coords['YDim'].values, "x": ds_coords['XDim'].values})
            
            if '_FillValue' in lst_data.attrs:
                lst_data = lst_data.where(lst_data != lst_data.attrs['_FillValue'])
            if 'scale_factor' in lst_data.attrs:
                lst_data = lst_data * lst_data.attrs['scale_factor']
            lst_celsius = lst_data - 273.15
            
            lst_celsius.rio.write_crs(SINUSOIDAL_PROJ, inplace=True)
            
            tarih_match = re.search(r'\.A(\d{7})\.', str(dosya_path))
            if not tarih_match: continue
            tarih = pd.to_datetime(tarih_match.group(1), format='%Y%j').strftime('%Y-%m-%d')
            
            iller_gdf_proj = iller_gdf.to_crs(lst_celsius.rio.crs)

            for index, il in iller_gdf_proj.iterrows():
                sehir_adi = il['sehir']
                try:
                    clipped = lst_celsius.rio.clip([il['geometry']], drop=True, all_touched=True)
                    ortalama_sicaklik = float(clipped.mean())
                    
                    if pd.notna(ortalama_sicaklik):
                        tum_sonuclar.append({"sehir": sehir_adi, "tarih": tarih, "sicaklik_C": ortalama_sicaklik})
                except Exception:
                    continue
        except Exception as e:
            print(f"\n⚠️ HATA: '{dosya_path.name}' atlanıyor. -> {e}")
            continue

    if not tum_sonuclar:
        print("\n❌ Test işlemi başarısız. Hiçbir veri işlenemedi.")
        return

    print("\nTest verileri işlendi. CSV dosyası oluşturuluyor...")
    sonuc_df = pd.DataFrame(tum_sonuclar)
    sonuc_df.to_csv(CIKTI_CSV_DOSYASI, index=False, encoding='utf-8')
    print(f"✨ Test Başarılı! Sonuçlar '{CIKTI_CSV_DOSYASI}' dosyasına kaydedildi.")
    print("\nCSV Dosyasının ilk 5 satırı:")
    print(sonuc_df.head())


if __name__ == '__main__':
    skorlari_hesapla_test()

# Sıcaklık Gece / Tempareture Night

In [ ]:
!pip install earthaccess

import earthaccess
from pathlib import Path
from datetime import datetime

# --- AYARLAR ---

# Veri Seti Bilgileri
VERI_SETI_KISA_ADI = 'VNP21A1N'
VERSIYON = '002'
YIL = 2021 # <-- Artık buraya yazdığın yıl ne ise, sadece o indirilecek.

# Türkiye'yi içine alan coğrafi kutu
TURKIYE_BOUNDING_BOX = (25, 35, 45, 43)

# Verilerin indirileceği klasör
INDIRME_KLASORU = f"NASA_Verileri/{VERI_SETI_KISA_ADI}/{YIL}"

# --- KOD BAŞLANGICI ---

print("NASA Earthdata sistemine giriş yapılıyor...")
try:
    auth = earthaccess.login()
    print("✅ Giriş başarılı.")
except Exception as e:
    print(f"❌ Giriş başarısız. Lütfen .netrc dosyasını kontrol edin. Hata: {e}")
    exit()

Path(INDIRME_KLASORU).mkdir(parents=True, exist_ok=True)
print(f"Dosyalar '{INDIRME_KLASORU}' klasörüne indirilecek.\n")

# === DÜZELTME BURADA: Bitiş tarihini YIL'a göre doğru ayarlayan mantık ===
baslangic_tarihi = f"{YIL}-01-01"
bugun = datetime.now()

if YIL < bugun.year:
    # Eğer geçmiş bir yılı arıyorsak, bitiş tarihi o yılın sonu (31 Aralık) olmalı.
    bitis_tarihi = f"{YIL}-12-31"
else:
    # Eğer içinde bulunduğumuz yılı (2025) arıyorsak, bitiş tarihi bugün olsun.
    bitis_tarihi = bugun.strftime("%Y-%m-%d")
# === DÜZELTME SONU ===

print(f"Veri Seti: {VERI_SETI_KISA_ADI}, Yıl: {YIL}")
print(f"Zaman Aralığı: {baslangic_tarihi} -> {bitis_tarihi}") # Bu satır artık doğru aralığı gösterecek
print(f"Bölge: Türkiye ({TURKIYE_BOUNDING_BOX})")
print("Veriler aranıyor...")

try:
    sonuclar = earthaccess.search_data(
        short_name=VERI_SETI_KISA_ADI,
        version=VERSIYON,
        temporal=(baslangic_tarihi, bitis_tarihi),
        bounding_box=TURKIYE_BOUNDING_BOX
    )
    
    if sonuclar:
        print(f"✅ {len(sonuclar)} adet dosya bulundu. İndirme işlemi başlıyor...")
        print("❗ Bu işlem uzun sürebilir, lütfen sabırlı olun...")
        
        earthaccess.download(sonuclar, local_path=INDIRME_KLASORU)
        
        print(f"\n✨ İndirme tamamlandı! {len(sonuclar)} dosya '{INDIRME_KLASORU}' klasörüne indirildi.")

    else:
        print("⚠ Belirtilen tarih ve bölge için hiç dosya bulunamadı.")

except Exception as e:
    print(f"❌ Arama veya indirme sırasında bir hata oluştu: {e}")

In [ ]:
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as rxr
from pathlib import Path
from tqdm import tqdm
import re
import numpy as np

# --- AYARLAR ---
# Test modunu aktive et ve işlenecek dosya sayısını belirle
TEST_MODU = False
TEST_DOSYA_SAYISI = 50 # Hızlı bir test için sadece 5 dosya

YIL = 2021
VERI_KLASORU = f"NASA_Verileri/VNP21A1N/{YIL}"
GEOJSON_DOSYASI = "tr-cities.json"

print(VERI_KLASORU)
# Çıktı dosya adını test moduna göre ayarla
if TEST_MODU:
    CIKTI_CSV_DOSYASI = f"sehir_bazli_sicaklik_skorlari_{YIL}_TEST.csv"
else:
    CIKTI_CSV_DOSYASI = f"sehir_bazli_sicaklik_skorlari_{YIL}.csv"

# h5 dosyası içindeki doğru grup ve değişken adları
HDF5_PARENT_GROUP = 'HDFEOS/GRIDS/VIIRS_Grid_Daily_1km_LST21'
HDF5_DATA_SUBGROUP = 'Data Fields'
HDF5_DEGISKENI = 'LST_1KM'

# NASA VIIRS/MODIS verilerinin kullandığı standart Sinusoidal Projeksiyon
SINUSOIDAL_PROJ = "+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs"

# --- KOD BAŞLANGICI ---

def skorlari_hesapla_test():
    iller_gdf = gpd.read_file(GEOJSON_DOSYASI)
    iller_gdf = iller_gdf[['name', 'geometry']].rename(columns={'name': 'sehir'})
    
    veri_yolu = Path(VERI_KLASORU)
    h5_dosyalari = sorted(list(veri_yolu.glob('*.h5')))
    
    # Test modu aktifse, dosya listesini kısalt
    if TEST_MODU:
        h5_dosyalari = h5_dosyalari[:TEST_DOSYA_SAYISI]
        print(f"🐞 TEST MODU AKTİF: Sadece ilk {len(h5_dosyalari)} dosya işlenecek.")
    
    print(f"İşlenmek üzere {len(h5_dosyalari)} adet sıcaklık dosyası seçildi.")
    print("İşlem başlıyor...")

    tum_sonuclar = []

    for dosya_path in tqdm(h5_dosyalari, desc="Günlük Sıcaklık Verileri İşleniyor"):
        try:
            data_group_path = f"{HDF5_PARENT_GROUP}/{HDF5_DATA_SUBGROUP}"
            ds_data = xr.open_dataset(dosya_path, engine='h5netcdf', group=data_group_path)
            ds_coords = xr.open_dataset(dosya_path, engine='h5netcdf', group=HDF5_PARENT_GROUP)

            lst_data = ds_data[HDF5_DEGISKENI]
            
            lst_data = lst_data.rename({lst_data.dims[0]: 'y', lst_data.dims[1]: 'x'})
            lst_data = lst_data.assign_coords({"y": ds_coords['YDim'].values, "x": ds_coords['XDim'].values})
            
            if '_FillValue' in lst_data.attrs:
                lst_data = lst_data.where(lst_data != lst_data.attrs['_FillValue'])
            if 'scale_factor' in lst_data.attrs:
                lst_data = lst_data * lst_data.attrs['scale_factor']
            lst_celsius = lst_data - 273.15
            
            lst_celsius.rio.write_crs(SINUSOIDAL_PROJ, inplace=True)
            
            tarih_match = re.search(r'\.A(\d{7})\.', str(dosya_path))
            if not tarih_match: continue
            tarih = pd.to_datetime(tarih_match.group(1), format='%Y%j').strftime('%Y-%m-%d')
            
            iller_gdf_proj = iller_gdf.to_crs(lst_celsius.rio.crs)

            for index, il in iller_gdf_proj.iterrows():
                sehir_adi = il['sehir']
                try:
                    clipped = lst_celsius.rio.clip([il['geometry']], drop=True, all_touched=True)
                    ortalama_sicaklik = float(clipped.mean())
                    
                    if pd.notna(ortalama_sicaklik):
                        tum_sonuclar.append({"sehir": sehir_adi, "tarih": tarih, "sicaklik_C": ortalama_sicaklik})
                except Exception:
                    continue
        except Exception as e:
            print(f"\n⚠️ HATA: '{dosya_path.name}' atlanıyor. -> {e}")
            continue

    if not tum_sonuclar:
        print("\n❌ Test işlemi başarısız. Hiçbir veri işlenemedi.")
        return

    print("\nTest verileri işlendi. CSV dosyası oluşturuluyor...")
    sonuc_df = pd.DataFrame(tum_sonuclar)
    sonuc_df.to_csv(CIKTI_CSV_DOSYASI, index=False, encoding='utf-8')
    print(f"✨ Test Başarılı! Sonuçlar '{CIKTI_CSV_DOSYASI}' dosyasına kaydedildi.")
    print("\nCSV Dosyasının ilk 5 satırı:")
    print(sonuc_df.head())


if __name__ == '__main__':
    skorlari_hesapla_test()

# Yağış / Precipitation

In [ ]:
# Gerekli kütüphaneyi kuralım
!pip install earthaccess -q

import earthaccess
from pathlib import Path
from datetime import datetime

# --- AYARLAR (YAĞIŞ VERİSİ İÇİN UYARLANDI) ---

# Veri Seti Bilgileri
VERI_SETI_KISA_ADI = 'GPM_3IMERGDF' # <-- GÜNCELLENDİ
VERSIYON = '07'                # <-- GÜNCELLENDİ
YIL = 2020                     # <-- İndirmek istediğin yılı buraya yaz

# Türkiye'yi içine alan coğrafi kutu
TURKIYE_BOUNDING_BOX = (25, 35, 45, 43)

# Verilerin indirileceği klasör
INDIRME_KLASORU = f"NASA_Verileri/{VERI_SETI_KISA_ADI}/{YIL}"

# --- KOD BAŞLANGICI (DEĞİŞİKLİK YOK) ---

print("NASA Earthdata sistemine giriş yapılıyor...")
try:
    auth = earthaccess.login()
    print("✅ Giriş başarılı.")
except Exception as e:
    print(f"❌ Giriş başarısız. Lütfen .netrc dosyasını kontrol edin. Hata: {e}")
    exit()

Path(INDIRME_KLASORU).mkdir(parents=True, exist_ok=True)
print(f"Dosyalar '{INDIRME_KLASORU}' klasörüne indirilecek.\n")

# Bitiş tarihini YIL'a göre doğru ayarlayan mantık
baslangic_tarihi = f"{YIL}-01-01"
bugun = datetime.now()

if YIL < bugun.year:
    bitis_tarihi = f"{YIL}-12-31"
else:
    bitis_tarihi = bugun.strftime("%Y-%m-%d")

print(f"Veri Seti: {VERI_SETI_KISA_ADI}, Yıl: {YIL}")
print(f"Zaman Aralığı: {baslangic_tarihi} -> {bitis_tarihi}")
print(f"Bölge: Türkiye ({TURKIYE_BOUNDING_BOX})")
print("Veriler aranıyor...")

try:
    sonuclar = earthaccess.search_data(
        short_name=VERI_SETI_KISA_ADI,
        version=VERSIYON,
        temporal=(baslangic_tarihi, bitis_tarihi),
        bounding_box=TURKIYE_BOUNDING_BOX
    )
    
    if sonuclar:
        print(f"✅ {len(sonuclar)} adet dosya bulundu. İndirme işlemi başlıyor...")
        print("❗ Bu işlem biraz zaman alabilir, lütfen sabırlı olun...")
        
        earthaccess.download(sonuclar, local_path=INDIRME_KLASORU)
        
        print(f"\n✨ İndirme tamamlandı! {len(sonuclar)} dosya '{INDIRME_KLASORU}' klasörüne indirildi.")

    else:
        print("⚠ Belirtilen tarih ve bölge için hiç dosya bulunamadı.")

except Exception as e:
    print(f"❌ Arama veya indirme sırasında bir hata oluştu: {e}")

In [ ]:
!pip install rioxarray

import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as rxr
from pathlib import Path
from tqdm import tqdm
import re
import numpy as np
import concurrent.futures
from shapely.geometry import box

# --- AYARLAR ---
# Test modunu aktive et ve işlenecek dosya sayısını belirle
TEST_MODU = False
TEST_DOSYA_SAYISI = 10

YIL = 2020

VERI_KLASORU = f"NASA_Verileri/GPM_3IMERGDF/{YIL}"
GEOJSON_DOSYASI = "tr-cities.json"

print(VERI_KLASORU)

# Çıktı dosya adını test moduna göre ayarla
if TEST_MODU:
    CIKTI_CSV_DOSYASI = f"sehir_bazli_yagis_skorlari_{YIL}_TEST.csv"
else:
    CIKTI_CSV_DOSYASI = f"sehir_bazli_yagis_skorlari_{YIL}.csv"

YAGIS_DEGISKENI = 'precipitation'
TURKIYE_BOUNDING_BOX = (25, 35, 45, 43)

# --- TEK BİR DOSYAYI İŞLEYEN FONKSİYON ---
def process_precipitation_file(dosya_path):
    try:
        ds = xr.open_dataset(dosya_path)
        yagis_data = ds[YAGIS_DEGISKENI]

        if 'time' in yagis_data.dims:
            yagis_data = yagis_data.squeeze('time', drop=True)

        yagis_data.rio.set_spatial_dims(x_dim='lon', y_dim='lat', inplace=True)
        yagis_data.rio.write_crs("epsg:4236", inplace=True)
        
        min_lon, min_lat, max_lon, max_lat = TURKIYE_BOUNDING_BOX
        turkey_geom = box(min_lon, min_lat, max_lon, max_lat)
        yagis_turkiye = yagis_data.rio.clip([turkey_geom], "epsg:4236")
        
        tarih_match = re.search(r'\.(\d{8})-S', str(dosya_path))
        if not tarih_match: return []
        tarih = pd.to_datetime(tarih_match.group(1), format='%Y%m%d').strftime('%Y-%m-%d')
        
        iller_gdf = gpd.read_file(GEOJSON_DOSYASI)
        iller_gdf = iller_gdf[['name', 'geometry']].rename(columns={'name': 'sehir'})
        iller_gdf_proj = iller_gdf.to_crs(yagis_data.rio.crs)
        
        gunluk_sonuclar = []
        for index, il in iller_gdf_proj.iterrows():
            sehir_adi = il['sehir']
            try:
                clipped = yagis_turkiye.rio.clip([il['geometry']], drop=True, all_touched=True)
                ortalama_yagis = float(clipped.mean())
                
                if pd.notna(ortalama_yagis):
                    gunluk_sonuclar.append({
                        "sehir": sehir_adi,
                        "tarih": tarih,
                        "yagis_mm_gun": ortalama_yagis
                    })
            except Exception:
                continue
        return gunluk_sonuclar
    except Exception as e:
        # print(f"HATA: {dosya_path.name} işlenemedi. Sebep: {e}")
        return []

# --- ANA KOD ---
if __name__ == '__main__':
    veri_yolu = Path(VERI_KLASORU)
    nc4_dosyalari = sorted(list(veri_yolu.glob('*.nc4')))
    
    # Test modu aktifse, dosya listesini kısalt
    if TEST_MODU:
        nc4_dosyalari = nc4_dosyalari[:TEST_DOSYA_SAYISI]
        print(f"🐞 TEST MODU AKTİF: Sadece ilk {len(nc4_dosyalari)} dosya işlenecek.")

    if not nc4_dosyalari:
        print(f"❌ HATA: '{veri_yolu}' klasöründe .nc4 dosyası bulunamadı.")
    else:
        print(f"İşlenmek üzere {len(nc4_dosyalari)} adet yağış dosyası seçildi.")
        print("Optimize edilmiş paralel işleme başlıyor...")

        tum_sonuclar = []
        with concurrent.futures.ProcessPoolExecutor() as executor:
            results_iterator = list(tqdm(executor.map(process_precipitation_file, nc4_dosyalari), total=len(nc4_dosyalari)))
        
        for result in results_iterator:
            tum_sonuclar.extend(result)

        if not tum_sonuclar:
            print("\n❌ İşlem bitti ancak hiçbir veri işlenemedi.")
        else:
            print("\nTest verileri işlendi. CSV dosyası oluşturuluyor...")
            sonuc_df = pd.DataFrame(tum_sonuclar)
            sonuc_df.to_csv(CIKTI_CSV_DOSYASI, index=False, encoding='utf-8')
            print(f"✨ Test Başarılı! Sonuçlar '{CIKTI_CSV_DOSYASI}' dosyasına kaydedildi.")
            print("\nCSV Dosyasının ilk 5 satırı:")
            print(sonuc_df.head())

# Model Eğitimi / Train Model

In [ ]:
!pip install pandas sklearn prophet

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from prophet import Prophet
import concurrent.futures
from tqdm import tqdm
import logging
import warnings
import os # Dosya yollarını birleştirmek için eklendi

# Prophet ve Stan'den gelen gereksiz log mesajlarını kapatalım
logging.getLogger('cmdstanpy').setLevel(logging.ERROR)
warnings.filterwarnings('ignore', category=FutureWarning)

# --- AYARLAR ---
# DEĞİŞİKLİK: Artık birden fazla yılı işliyoruz
YILLAR = [2020, 2021, 2022, 2023, 2024,2025] 
ANA_KLASOR = "/content/drive/MyDrive/Nasa/Tum_veri/" # Ana dosya yolu

# Tahmin edilecek özel tarihler
TAHMIN_TARIHLERI = ['2027-01-15', '2027-04-15', '2027-07-15', '2027-10-15']

# Çıktı dosyalarının adları
YIL_ARALIGI_STR = f"{YILLAR[0]}-{YILLAR[-1]}"
FINAL_DAILY_CSV = f"EcoPulse_Skorlari_GUNLUK_{YIL_ARALIGI_STR}_Temizlenmis.csv"
FINAL_MONTHLY_CSV = f"EcoPulse_Skorlari_AYLIK_{YIL_ARALIGI_STR}_Temizlenmis.csv" # YENİ ÇIKTI
FINAL_FORECAST_CSV = f"EcoPulse_2027_Tahminleri_{YIL_ARALIGI_STR}_Verisiyle.csv"

# --- 1. Adım: Tüm Yıllar İçin Veri Hazırlama ve TEMİZLEME ---
def cok_yilli_veri_hazirla():
    print(f"{YIL_ARALIGI_STR} yılları arasındaki tüm veriler birleştiriliyor...")
    
    tum_datalar = []
    for yil in YILLAR:
        try:
            # Her yıl için dosya yollarını dinamik olarak oluştur
            no2_csv = os.path.join(ANA_KLASOR, f"hava_kirliligi/sehir_bazli_no2_skorlari_{yil}.csv")
            gece_temp_csv = os.path.join(ANA_KLASOR, f"sıcaklık_gece/sehir_bazli_sicaklik_skorlari_{yil}.csv")
            gunduz_temp_csv = os.path.join(ANA_KLASOR, f"sıcaklık_gunduz/sehir_bazli_sicaklik_skorlari_{yil}_gunduz.csv")
            yagis_csv = os.path.join(ANA_KLASOR, f"yagıs/sehir_bazli_yagis_skorlari_{yil}.csv")

            df_no2 = pd.read_csv(no2_csv)
            df_gece_temp = pd.read_csv(gece_temp_csv)
            df_gunduz_temp = pd.read_csv(gunduz_temp_csv)
            df_yagis = pd.read_csv(yagis_csv)
            
            df_gece_temp = df_gece_temp.rename(columns={'sicaklik_C': 'sicaklik_gece_C'})
            df_gunduz_temp = df_gunduz_temp.rename(columns={'sicaklik_C': 'sicaklik_gunduz_C'})
            
            df_merged = pd.merge(df_no2, df_gece_temp, on=['sehir', 'tarih'])
            df_merged = pd.merge(df_merged, df_gunduz_temp, on=['sehir', 'tarih'])
            df_merged = pd.merge(df_merged, df_yagis, on=['sehir', 'tarih'])
            
            tum_datalar.append(df_merged)
            print(f"✅ {yil} yılı verileri başarıyla okundu.")

        except FileNotFoundError as e:
            print(f"❌ UYARI: {yil} yılı için bir veya daha fazla dosya bulunamadı. Bu yıl atlanıyor. Hata: {e}")
            continue

    if not tum_datalar:
        print("❌ HATA: Hiçbir yıla ait veri okunamadı. Lütfen dosya yollarını ve adlarını kontrol edin.")
        return None

    # Tüm yılların verilerini tek bir dataframe'de birleştir
    df_final = pd.concat(tum_datalar, ignore_index=True)
    df_final['tarih'] = pd.to_datetime(df_final['tarih'])
    
    # --- VERİ TEMİZLEME ADIMI ---
    print("\nVeri temizleme adımı başlatılıyor...")
    orijinal_satir_sayisi = len(df_final)
    MIN_SICAKLIK, MAX_SICAKLIK = -25.0, 55.0
    
    df_final = df_final[
        (df_final['sicaklik_gece_C'].between(MIN_SICAKLIK, MAX_SICAKLIK)) &
        (df_final['sicaklik_gunduz_C'].between(MIN_SICAKLIK, MAX_SICAKLIK))
    ]
    
    kaldirilan_satir_sayisi = orijinal_satir_sayisi - len(df_final)
    print(f"✅ Veri temizleme tamamlandı. {kaldirilan_satir_sayisi} adet hatalı satır kaldırıldı.")
    
    # --- GÜNLÜK ECOPULSE HESAPLAMA ---
    print("\nGünlük EcoPulse skorları hesaplanıyor...")
    df_final['sicaklik_farki_DTR'] = df_final['sicaklik_gunduz_C'] - df_final['sicaklik_gece_C']
    
    scaler = MinMaxScaler()
    df_final['P_norm'] = (1 - scaler.fit_transform(df_final[['no2_skoru']])) * 100
    df_final['H_norm'] = (1 - scaler.fit_transform(df_final[['sicaklik_gece_C']])) * 100
    df_final['R_norm'] = scaler.fit_transform(df_final[['yagis_mm_gun']]) * 100
    df_final['DTR_norm'] = (1 - scaler.fit_transform(df_final[['sicaklik_farki_DTR']])) * 100
    
    df_final['EcoPulse'] = (0.30 * df_final['P_norm']) + (0.30 * df_final['DTR_norm']) + \
                           (0.20 * df_final['R_norm']) + (0.20 * df_final['H_norm'])
    print("✅ Günlük EcoPulse skorları hazır.")
    
    return df_final

# --- Model Fonksiyonu (Değişiklik yok) ---
def train_and_forecast_city_specific_dates(args):
    # ... (Bu fonksiyon önceki kod ile aynı, değişiklik yok)
    sehir_adi, df_sehir, dates_to_predict = args
    try:
        if len(df_sehir) < 2: return []
        df_prophet = df_sehir[['tarih', 'EcoPulse']].rename(columns={'tarih': 'ds', 'EcoPulse': 'y'})
        df_prophet['cap'], df_prophet['floor'] = 100, 0
        model = Prophet(growth='logistic', yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
        model.fit(df_prophet)
        future = pd.DataFrame({'ds': dates_to_predict}); future['cap'], future['floor'] = 100, 0
        forecast = model.predict(future)
        return [{"sehir": sehir_adi, "tahmin_tarihi": row['ds'].strftime('%Y-%m-%d'), "tahmini_ecopulse_skoru": row['yhat']} for _, row in forecast.iterrows()]
    except Exception as e:
        print(f"HATA: {sehir_adi} şehri için tahmin yapılırken bir sorun oluştu: {e}")
        return []

# --- ANA KOD ---
if __name__ == '__main__':
    df_gunluk_ecopulse = cok_yilli_veri_hazirla()
    
    if df_gunluk_ecopulse is not None and not df_gunluk_ecopulse.empty:
        # --- GÜNLÜK VERİYİ KAYDET ---
        print(f"\nGünlük hesaplanan EcoPulse skorları '{FINAL_DAILY_CSV}' dosyasına kaydediliyor...")
        kolonlar = ['sehir', 'tarih', 'EcoPulse', 'no2_skoru', 'sicaklik_gece_C', 'sicaklik_gunduz_C', 'yagis_mm_gun']
        df_gunluk_ecopulse[kolonlar].to_csv(FINAL_DAILY_CSV, index=False, encoding='utf-8', float_format='%.2f')
        print(f"✅ Günlük veriler başarıyla kaydedildi.")

        # --- YENİ ADIM: AYLIK ORTALAMALARI HESAPLA VE KAYDET ---
        print(f"\nAylık ortalama EcoPulse skorları '{FINAL_MONTHLY_CSV}' dosyasına kaydediliyor...")
        df_aylik = df_gunluk_ecopulse.groupby(['sehir', pd.Grouper(key='tarih', freq='M')])['EcoPulse'].mean().reset_index()
        df_aylik = df_aylik.rename(columns={'EcoPulse': 'aylik_ortalama_ecopulse'})
        df_aylik['tarih'] = df_aylik['tarih'].dt.to_period('M')
        df_aylik['aylik_ortalama_ecopulse'] = df_aylik['aylik_ortalama_ecopulse'].round(2)
        df_aylik.to_csv(FINAL_MONTHLY_CSV, index=False, encoding='utf-8')
        print(f"✅ Aylık ortalama verileri başarıyla kaydedildi.")
        print("Aylık verilerden örnek:")
        print(df_aylik.head())
        # --- AYLIK ORTALAMA ADIMI BİTTİ ---

        # --- MODEL EĞİTİMİ VE TAHMİN ---
        predict_dates = pd.to_datetime(TAHMIN_TARIHLERI)
        sehirler = df_gunluk_ecopulse['sehir'].unique()
        sehir_datalari = [(sehir, df_gunluk_ecopulse[df_gunluk_ecopulse['sehir'] == sehir], predict_dates) for sehir in sehirler]
        
        print(f"\n{len(sehirler)} şehir için paralel olarak {len(predict_dates)} özel gün tahmini yapılıyor...")
        
        tum_tahminler = []
        with concurrent.futures.ProcessPoolExecutor() as executor:
            results = list(tqdm(executor.map(train_and_forecast_city_specific_dates, sehir_datalari), total=len(sehir_datalari)))

        for res in results: tum_tahminler.extend(res)
        
        if tum_tahminler:
            print("\n✅ Tüm şehirler için özel gün tahminleri tamamlandı.")
            df_tahmin = pd.DataFrame(tum_tahminler)
            df_tahmin['tahmini_ecopulse_skoru'] = df_tahmin['tahmini_ecopulse_skoru'].clip(0, 100).round(2)
            df_tahmin.to_csv(FINAL_FORECAST_CSV, index=False, encoding='utf-8')
            print(f"✨ Tahmin sonuçları '{FINAL_FORECAST_CSV}' dosyasına kaydedildi.")
        else:
            print("\n❌ Tahmin üretilemedi.")
    else:
        print("\n❌ Hiç geçerli veri kalmadığı için hiçbir işlem yapılamadı.")

In [ ]:
import pandas as pd
import numpy as np
import os

# --- AYARLAR ---
# Lütfen ana kodun ürettiği AYLIK ortalama dosyasının adının
# bu olduğundan emin ol. Yıl aralığı farklıysa (örn: 2020-2024),
# bu değişkeni güncelleyebilirsin.
YIL_ARALIGI_STR = "2020-2025" 
AYLIK_ORTALAMA_DOSYASI = f"EcoPulse_Skorlari_AYLIK_{YIL_ARALIGI_STR}_Temizlenmis.csv"

def kategori_sinirlarini_bul():
    """
    Aylık EcoPulse skorlarını analiz eder ve web sitesinde kullanılacak
    Kırmızı, Sarı, Yeşil renk kategorileri için en uygun sınırları önerir.
    """
    # Dosyanın var olup olmadığını kontrol et
    if not os.path.exists(AYLIK_ORTALAMA_DOSYASI):
        print(f"❌ HATA: '{AYLIK_ORTALAMA_DOSYASI}' dosyası bulunamadı.")
        print("Lütfen dosya adının doğru olduğundan ve bu kodun ilgili dosyayla aynı klasörde olduğundan emin ol.")
        return

    print(f"'{AYLIK_ORTALAMA_DOSYASI}' dosyası okunuyor ve analiz ediliyor...")
    
    # 1. Adım: Veriyi oku
    df_aylik = pd.read_csv(AYLIK_ORTALAMA_DOSYASI)
    skor_kolonu = 'aylik_ortalama_ecopulse'

    # 2. Adım: Verinin genel istatistiksel dağılımını göster
    # Bu, verinin genel yapısını anlamamızı sağlar (min, max, ortalama vb.)
    print("\n--- Aylık EcoPulse Skorlarının Genel Dağılımı ---")
    distribution_stats = df_aylik[skor_kolonu].describe()
    print(distribution_stats)
    
    # 3. Adım: Yüzdelik dilim (percentile) yöntemini kullanarak sınırları hesapla
    # Bu yöntem, veri setini 3 eşit parçaya böler.
    # Her renk kategorisinin veri setinde anlamlı sayıda örnek içermesini sağlar.
    try:
        sinir_kirmizi_sari = np.percentile(df_aylik[skor_kolonu], 33)
        sinir_sari_yesil = np.percentile(df_aylik[skor_kolonu], 66)
        
        min_skor = distribution_stats['min']
        max_skor = distribution_stats['max']

        print("\n" + "="*50)
        print("📊 WEB SİTESİ İÇİN RENK KATEGORİSİ ÖNERİLERİ 📊")
        print("="*50)
        
        print(f"\nSenin ilk önerin:")
        print(f"🔴 Kırmızı: 0 - 35")
        print(f"🟡 Sarı   : 35 - 65")
        print(f"🟢 Yeşil  : 65 - 100")
        
        # Sonuçları daha temiz ve kullanılabilir hale getirelim (tam sayılara yuvarlayarak)
        alt_sinir_sari = int(round(sinir_kirmizi_sari))
        alt_sinir_yesil = int(round(sinir_sari_yesil))
        
        print(f"\n📈 Veri Dağılımına Göre Önerilen Yeni Aralıklar:")
        print(f"🔴 Kırmızı (Düşük Performans): 0 - {alt_sinir_sari}")
        print(f"🟡 Sarı   (Orta Performans)  : {alt_sinir_sari} - {alt_sinir_yesil}")
        print(f"🟢 Yeşil  (İyi Performans)   : {alt_sinir_yesil} - 100")
        print("="*50)
        print("\nBu aralıklar, veri setindeki skorları yaklaşık 3 eşit gruba ayırır.")

    except Exception as e:
        print(f"\nHesaplama sırasında bir hata oluştu: {e}")

# Ana fonksiyonu çalıştır
if __name__ == '__main__':
    kategori_sinirlarini_bul()